d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Packaging ML Projects

Machine learning projects need to produce both reusable code and reproducible results.  This lesson examines creating, organizing, and packaging machine learning projects with a focus on reproducibility and collaborating with a team.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Introduce organizing code into projects
 - Package a basic project with parameters and an environment
 - Run a basic project locally and remotely using Github

## Prerequisites
- Web browser: Chrome
- A cluster configured with **8 cores** and **DBR 6.2**

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the<br/>
start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [4]:
%run "./Includes/Classroom-Setup"

<iframe  
src="//fast.wistia.net/embed/iframe/2q71x71wrc?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/2q71x71wrc?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
### The Case for Packaging

There are a number of different reasons why teams need to package their machine learning projects:<br><br>

1. Projects have various library dependencies so shipping a machine learning solution involves the environment in which it was built.  MLflow allows for this environment to be a conda environment or docker container.  This means that teams can easily share and publish their code for others to use.
2. Machine learning projects become increasingly complex as time goes on.  This includes ETL and featurization steps, machine learning models used for pre-processing, and finally the model training itself.
3. Each component of a machine learning pipeline needs to allow for tracing its lineage.  If there's a failure at some point, tracing the full end-to-end lineage of a model allows for easier debugging.

**ML Projects is a specification for how to organize code in a project.**  The heart of this is an **MLproject file,** a YAML specification for the components of an ML project.  This allows for more complex workflows since a project can execute another project, allowing for encapsulation of each stage of a more complex machine learning architecture.  This means that teams can collaborate more easily using this architecture.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/mlflow-project.png" style="height: 400px; margin: 20px"/></div>

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See <a href="https://github.com/mlflow/mlflow-example" target="_blank">this example project backed by GitHub</a> for an example of how to integrate MLflow with source control.

### Packaging a Simple Project

First we're going to create a simple MLflow project consisting of the following elements:<br><br>

1. MLProject file
2. Conda environment
3. Basic machine learning script

We're going to want to be able to pass parameters into this code so that we can try different hyperparameter options.

Create a new experiment for this exercise.  Navigate to the UI in another tab.

In [9]:
experimentPath = "/Users/" + username + "/experiment-SPL3"

In [10]:
import mlflow

mlflow.set_experiment(experimentPath)
mlflow_client = mlflow.tracking.MlflowClient()
experimentID = mlflow_client.get_experiment_by_name(name=experimentPath).experiment_id

print(f"The experiment can be found at the path `{experimentPath}` and has an experiment_id of `{experimentID}`")

INFO: '/Users/tbresee@umich.edu/experiment-SPL3' does not exist. Creating a new experiment
The experiment can be found at the path `/Users/tbresee@umich.edu/experiment-SPL3` and has an experiment_id of `1243748146715256`

-sandbox
First, examine the code we're going to run.  This looks similar to what we ran in the last lesson with the addition of decorators from the `click` library.  This allows us to parameterize our code.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We'll uncomment out the `__main__` block when we save this code as a Python file.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Check out the <a href="https://click.palletsprojects.com/en/7.x/" target="_blank">`click` docs here.</a>

In [12]:
import click
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

@click.command()
@click.option("--data_path", default="/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv", type=str)
@click.option("--n_estimators", default=10, type=int)
@click.option("--max_depth", default=20, type=int)
@click.option("--max_features", default="auto", type=str)
def mlflow_rf(data_path, n_estimators, max_depth, max_features):

  with mlflow.start_run() as run:
    # Import the data
    df = pd.read_csv(data_path)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)
    
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")
    
    # Log params
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("max_features", max_features)

    # Log metrics
    mlflow.log_metric("mse", mean_squared_error(y_test, predictions))
    mlflow.log_metric("mae", mean_absolute_error(y_test, predictions))  
    mlflow.log_metric("r2", r2_score(y_test, predictions))  

# if __name__ == "__main__":
#   mlflow_rf() # Note that this does not need arguments thanks to click

Test that it works using the `click` `CliRunner`, which will execute the code in the same way we expect to.

In [14]:
from click.testing import CliRunner

runner = CliRunner()
result = runner.invoke(mlflow_rf, ['--n_estimators', 10, '--max_depth', 20], catch_exceptions=True)

assert result.exit_code == 0, "Code failed" # Check to see that it worked

print("Success!")

Success!

Now create a directory to hold our project files.  This will be a unique directory for this lesson.

In [16]:
# Adust our working directory from what DBFS sees to what python actually sees
working_path = workingDir.replace("dbfs:", "/dbfs")

-sandbox
Create the `MLproject` file.  This is the heart of an MLflow project.  It includes pointers to the conda environment and a `main` entry point, which is backed by the file `train.py`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Any `.py` or `.sh` file can be an entry point.

In [18]:
dbutils.fs.put(f"{workingDir}/MLproject", 
'''
name: Lesson-3-Model-Training

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      data_path: {type: str, default: "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv"}
      n_estimators: {type: int, default: 10}
      max_depth: {type: int, default: 20}
      max_features: {type: str, default: "auto"}
    command: "python train.py --data_path {data_path} --n_estimators {n_estimators} --max_depth {max_depth} --max_features {max_features}"
'''.strip())

Wrote 471 bytes.
Out[18]: True

-sandbox
Create the conda environment.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You can also dynamically view and use a package version by calling `.__version__` on the package.

In [20]:
import cloudpickle, numpy, pandas, sklearn

file_contents = f"""
name: Lesson-03
channels:
  - defaults
dependencies:
  - cloudpickle={cloudpickle.__version__}
  - numpy={numpy.__version__}
  - pandas={pandas.__version__}
  - scikit-learn={sklearn.__version__}
  - pip:
    - mlflow=={mlflow.__version__}
""".strip()

dbutils.fs.put(f"{workingDir}/conda.yaml", file_contents, overwrite=True)

print(file_contents)

Wrote 162 bytes.
name: Lesson-03
channels:
 - defaults
dependencies:
 - cloudpickle=1.3.0
 - numpy=1.16.2
 - pandas=0.24.2
 - scikit-learn=0.20.3
 - pip:
 - mlflow==1.8.0

Now create the code itself.  This is the same as above except for with the `__main__` is included.  Note how there are no arguments passed into `mlflow_rf()` on the final line.  `click` is handling the arguments for us.

In [22]:
dbutils.fs.put(f"{workingDir}/train.py", 
'''
import click
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

@click.command()
@click.option("--data_path", default="/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv", type=str)
@click.option("--n_estimators", default=10, type=int)
@click.option("--max_depth", default=20, type=int)
@click.option("--max_features", default="auto", type=str)
def mlflow_rf(data_path, n_estimators, max_depth, max_features):

  with mlflow.start_run() as run:
    # Import the data
    df = pd.read_csv(data_path)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)
    
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")
    
    # Log params
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("max_features", max_features)

    # Log metrics
    mlflow.log_metric("mse", mean_squared_error(y_test, predictions))
    mlflow.log_metric("mae", mean_absolute_error(y_test, predictions))  
    mlflow.log_metric("r2", r2_score(y_test, predictions))  

if __name__ == "__main__":
  mlflow_rf() # Note that this does not need arguments thanks to click
'''.strip(), True)

Wrote 1610 bytes.
Out[20]: True

To summarize, you now have three files: `MLproject`, `conda.yaml`, and `train.py`

In [24]:
display( dbutils.fs.ls(workingDir) )

path,name,size
dbfs:/user/tbresee@umich.edu/mlflow/03_packaging_ml_projects_psp/MLproject,MLproject,471
dbfs:/user/tbresee@umich.edu/mlflow/03_packaging_ml_projects_psp/conda.yaml,conda.yaml,162
dbfs:/user/tbresee@umich.edu/mlflow/03_packaging_ml_projects_psp/train.py,train.py,1610


-sandbox
### Running Projects

Now you have the three files we need to run the project, we can trigger the run.  We'll do this in a few different ways:<br><br>

1. On the driver node of our Spark cluster
2. On a new Spark cluster submitted as a job
3. Using files backed by GitHub

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> This currently relies on environment variables.  [See the setup script for details.]($./Includes/MLflow)

-sandbox
Now run the experiment.  This command will execute against the driver node of a Spark cluster, though it could be running locally or on a different remote VM.

First set the experiment using the `experimentPath` defined earlier.  Prepend `/dbfs` to the file path, which allows the cluster's file system to access DBFS.  Then, pass your parameters.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> This will take a few minutes to build the environment for the first time.  Subsequent runs are faster since `mlflow` can reuse the same environment after it has been built.

In [27]:
import mlflow

mlflow.projects.run(uri=working_path,
  parameters={
    "data_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv",
    "n_estimators": 10,
    "max_depth": 20,
    "max_features": "auto"
})

Out[22]: <mlflow.projects.submitted_run.LocalSubmittedRun at 0x7f011ae5beb8>

Check the run in the UI.  Notice that you can see the run command.  **This is very helpful in debugging.**

Now that it's working, experiment with other parameters.

In [29]:
mlflow.projects.run(working_path,
  parameters={
    "data_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv",
    "n_estimators": 1000,
    "max_depth": 10,
    "max_features": "log2"
})

Out[26]: <mlflow.projects.submitted_run.LocalSubmittedRun at 0x7f011abb4ef0>

How did the new model do?

-sandbox
Now try executing this code against a new Databricks cluster.  This needs cluster specifications in order for Databricks to know what kind of cluster to use.  Uncomment the following to run it.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/>  <a href="https://docs.databricks.com/api/latest/clusters.html" target="_blank">See the clusters API docs</a> to see how to define cluster specifications.

In [32]:
### issue with   backend_config=clusterspecs ? 

In [33]:
clusterspecs = {
    "num_workers": 2,
    "spark_version": "5.3.x-cpu-ml-scala2.11",
    "node_type_id": "Standard_DS3_v2",
    "driver_node_type_id": "Standard_DS3_v2",
}

mlflow.projects.run(
  uri=working_path,
  parameters={
    "data_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv",
    "n_estimators": 1500,
    "max_depth": 5,
    "max_features": "sqrt"
},
  backend="databricks",
  backend_config=clusterspecs
)

--------------------------------------------------------------------------- 
 MlflowException Traceback (most recent call last)
 <command-1243748146714023> in <module> 
 15 },
 16 backend = "databricks" , 
 ---> 17 backend_config = clusterspecs
 18 )

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/__init__.py in run (uri, entry_point, version, parameters, docker_args, experiment_name, experiment_id, backend, backend_config, use_conda, storage_dir, synchronous, run_id) 
 289 parameters = parameters , docker_args = docker_args , backend = backend , 
 290 backend_config = cluster_spec_dict , use_conda = use_conda , storage_dir = storage_dir , 
 --> 291 synchronous=synchronous, run_id=run_id)
 292 if synchronous : 
 293 _wait_for ( submitted_run_obj ) 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/__init__.py in _run (uri, experiment_id, entry_point, version, parameters, docker_args, backend, backend_config, use_conda, storage_dir, synchronous, run_id) 
 136 remote_run = active_run , 
 137 uri = uri , entry_point = entry_point , work_dir = work_dir , parameters = parameters , 
 --> 138 experiment_id=experiment_id, cluster_spec=backend_config)
 139 
 140 elif backend == "local" or backend is None : 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/databricks.py in run_databricks (remote_run, uri, entry_point, work_dir, parameters, experiment_id, cluster_spec) 
 286 db_job_runner = DatabricksJobRunner ( databricks_profile = profile ) 
 287 db_run_id = db_job_runner.run_databricks(
 --> 288 uri, entry_point, work_dir, parameters, experiment_id, cluster_spec, run_id)
 289 submitted_run = DatabricksSubmittedRun ( db_run_id , run_id , db_job_runner ) 
 290 submitted_run . _print_description_and_log_tags ( ) 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/databricks.py in run_databricks (self, uri, entry_point, work_dir, parameters, experiment_id, cluster_spec, run_id) 
 196 # Launch run on Databricks 
 197 command = _get_databricks_run_cmd ( dbfs_fuse_uri , run_id , entry_point , parameters ) 
 --> 198 return self . _run_shell_command_job ( uri , command , env_vars , cluster_spec ) 
 199 
 200 def _get_status ( self , databricks_run_id ) : 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/databricks.py in _run_shell_command_job (self, project_uri, command, env_vars, cluster_spec) 
 181 "libraries" : [ { "pypi" : { "package" : "'mlflow<=%s'" % VERSION } } ] , 
 182 }
 --> 183 run_submit_res = self . _jobs_runs_submit ( req_body_json ) 
 184 databricks_run_id = run_submit_res [ "run_id" ] 
 185 return databricks_run_id

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/databricks.py in _jobs_runs_submit (self, req_body) 
 78 def _jobs_runs_submit ( self , req_body ) : 
 79 response = self._databricks_api_request(
 ---> 80 endpoint="/api/2.0/jobs/runs/submit", method="POST", json=req_body)
 81 return json . loads ( response . text ) 
 82 

 /databricks/python/lib/python3.7/site-packages/mlflow/projects/databricks.py in _databricks_api_request (self, endpoint, method, **kwargs) 
 74 host_creds = databricks_utils . get_databricks_host_creds ( self . databricks_profile ) 
 75 return rest_utils.http_request_safe(
 ---> 76 host_creds=host_creds, endpoint=endpoint, method=method, **kwargs)
 77 
 78 def _jobs_runs_submit ( self , req_body ) : 

 /databricks/python/lib/python3.7/site-packages/mlflow/utils/rest_utils.py in http_request_safe (host_creds, endpoint, **kwargs) 
 93 Wrapper around ` ` http_request ` ` that also verifies that the request succeeds with code 200. 
 94 """
 ---> 95 response = http_request ( host_creds = host_creds , endpoint = endpoint , ** kwargs ) 
 96 return verify_rest_response ( response , endpoint ) 
 97 

 /databricks/python/lib/python3.7/site-packages/mlflow/utils/rest_utils.py in http_request (host_creds, endpoint, retries, retry_interval, max_rate_limit_interval, **kwargs) 
 78 time . sleep ( retry_interval ) 
 79 r

Finally, run this example, which is <a href="https://github.com/mlflow/mlflow-example" target="_blank">a project backed by GitHub.</a>

In [35]:
mlflow.run(
  uri="https://github.com/mlflow/mlflow-example",
  parameters={'alpha':0.4}
)

Out[28]: <mlflow.projects.submitted_run.LocalSubmittedRun at 0x7f011a403cc0>

## Review

**Question:** Why is packaging important?  
**Answer:** Packaging not only manages your code but the environment in which it was run.  This environment can be a Conda or Docker environment.  This ensures that you have reproducible code and models that can be used in a number of downstream environments.

**Question:** What are the core components of MLflow projects?  
**Answer:** An MLmodel specifies the project components using YAML.  The environment file contains specifics about the environment.  The code itself contains the steps to create a model or process data.

**Question:** What code can I run and where can I run it?  
**Answer:** Arbitrary code can be run in any number of different languages.  It can be run locally or remotely, whether on a remote VM, Spark cluster, or submitted as a Databricks job.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [41]:
%run "./Includes/Classroom-Cleanup"

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Next Steps

Start the labs for this lesson, [Packaging ML Projects Lab]($./Labs/03-Lab)

## Additional Topics & Resources

**Q:** Where can I find out more information on MLflow Projects?  
**A:** Check out the <a href="https://www.mlflow.org/docs/latest/projects.html" target="_blank">MLflow docs</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>